# Cluster differential expressed genes

In [ ]:
library_load <- suppressMessages(
    
    list(
        
        # Seurat 
        library(Seurat), 
        
        # Data 
        library(tidyverse), 
        library(openxlsx), 
        
        # Plotting 
        library(patchwork)
        
    )
)

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Source files
source("plotting_global.R")
source("bin/SeuratDEG.R")

In [ ]:
# Plotting Theme
ggplot2::theme_set(theme_global_set()) # From project global source()

In [ ]:
options(warn=-1)

# Parameter settings

In [ ]:
so_file <- "data/object/seurat_int.rds"

# Import Seurat 

In [ ]:
so <- readRDS(so_file)

# Cluster annotation 

In [ ]:
ref <- readRDS("data/haemosphere/se_haemosphere.rds")

In [ ]:
source("bin/SingleRQC.R")
so <- SingleRSeurat(so, ref=ref, cluster="integrated_snn_res.0.8")

In [ ]:
so$main_labels_cluster <- factor(so$main_labels_cluster, factor(names(color$main_labels_haemosphere)))
so$fine_labels_cluster <- factor(so$fine_labels_cluster, factor(names(color$fine_labels_haemosphere)))

# Compute cluster differential expressed genes 

In [ ]:
Idents(so) <- "treatment"
so <- SplitObject(so, split.by="integrated_snn_res.0.8")
so <- so[as.character(1:length(so))]

In [ ]:
deg <- list()
for(i in names(so)) {
    
    result <- FindMarkers(so[[i]], ident.1="CpG", ident.2="NaCl", assay="RNA", slot="data", test.use="wilcox", verbose=TRUE)
    result$cluster <- i
    deg[[i]] <- result
    
}

In [ ]:
saveRDS(deg, "result/treatment_deg/treatment_deg.rds")

In [ ]:
deg <- deg[as.character(1:length(deg))]
names(deg) <- paste0("cluster_", names(deg))

## Export xlsx 

In [ ]:
write.xlsx(deg, file="result/treatment_deg/treatment_deg.xlsx", overwrite=TRUE)

# Vulcaon plots

In [ ]:
source("bin/SeuratQC.R")
vp <- lapply(deg, deg_volcano_plot, log2_thold=0.25, adjpvalue_thold=0.05)

In [ ]:
options(repr.plot.width=25, repr.plot.height=25)
wrap_plots(vp)

# Heat maps

## Average expression 

In [ ]:
av_exp <- lapply(so, AverageExpression, assay="RNA", slot="data", group.by="treatment")
av_exp <- lapply(av_exp, function(x) x[["RNA"]])

## Increased expression in CpG group 

In [ ]:
source("bin/SeuratDEG.R")
av_exp_hm <- mapply(average_expression_hm, deg, av_exp, scale="row", avg_log2FC="pos", SIMPLIFY=FALSE)

In [ ]:
options(repr.plot.width=25, repr.plot.height=15)
av_exp_grid <- gridExtra::arrangeGrob(grobs=av_exp_hm, ncol=10)
grid::grid.draw(av_exp_grid)

## Decreased expression in CpG group

In [ ]:
source("bin/SeuratDEG.R")
av_exp_hm <- mapply(average_expression_hm, deg, av_exp, scale="row", avg_log2FC="neg", SIMPLIFY=FALSE)

In [ ]:
options(repr.plot.width=25, repr.plot.height=15)
av_exp_grid <- gridExtra::arrangeGrob(grobs=av_exp_hm, ncol=10)
grid::grid.draw(av_exp_grid)